# Backtesting 5 EMA

In [1]:
import pandas as pd
from datetime import datetime
import ta
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Disclaimer: 
The trading strategy and backtest results mentioned in this post are provided for educational purposes only and should not be considered as financial advice. The primary purpose is to showcase the use of Python for analyzing financial data and not to provide specific strategic guidance for trading purposes. Trading in the stock market involves inherent risks, and past performance is not indicative of future results. Before making any investment decisions, it is crucial to conduct your own thorough research, seek professional advice, and carefully assess your financial situation and risk tolerance. The author of this post cannot guarantee the accuracy or correctness of the code used for backtesting, and there may be potential bugs or errors that could result in incorrect information. Therefore, readers should exercise caution and due diligence when engaging in trading or investing activities. The author shall not be held responsible for any losses or damages incurred as a result of using or implementing the strategy mentioned.

# Importing the Data Set

In [2]:
data_path = r"D:\BackupFolder\Python Concepts\Backtestig_ 5EMA\NIFTY 50_minute_data_with_indicators.csv\NIFTY 50_minute_data_with_indicators.csv"

In [3]:
data = pd.read_csv(data_path)

In [4]:
MAX_SHORT_SL_POINTS = 6
MAX_LONG_SL_POINTS = 12

In [5]:
data.head()

,date,close,high,low,open,volume,sma5,sma10,sma15,sma20,...,fastd,fastksr,fastdsr,ULTOSC,WILLR,ATR,Trange,TYPPRICE,HT_DCPERIOD,BETA
0,2015-01-09 10:18:00+05:30,8283.35,8283.35,8279.40,8280.70,0,8281.88,8280.065,8281.386667,8282.1400,...,67.108912,100.0,73.853949,56.367315,-26.637555,3.652663,3.95,8282.033333,29.911385,1.500701
1,2015-01-09 10:19:00+05:30,8283.35,8283.70,8281.05,8283.05,0,8282.24,8280.370,8281.243333,8282.2075,...,75.198413,100.0,84.545206,59.482179,-26.637555,3.581045,2.65,8282.700000,29.577116,1.727908
2,2015-01-09 10:20:00+05:30,8283.95,8284.15,8283.10,8283.65,0,8282.49,8280.840,8281.273333,8282.1875,...,88.888889,100.0,100.000000,59.147298,-15.492958,3.400256,1.05,8283.733333,29.267897,2.251841
3,2015-01-09 10:21:00+05:30,8285.75,8285.75,8283.60,8283.75,0,8283.47,8281.840,8281.303333,8282.2225,...,93.650794,100.0,100.000000,59.852782,-0.000000,3.310952,2.15,8285.033333,29.026773,1.098915
4,2015-01-09 10:22:00+05:30,8286.80,8287.70,8285.75,8286.20,0,8284.64,8282.815,8281.556667,8282.2575,...,94.798241,100.0,100.000000,61.366844,-7.058824,3.213741,1.95,8286.750000,28.623841,0.863321


In [6]:
data = data[["date", "open", "high", "low", "close"]]

In [7]:
data.head()

,date,open,high,low,close
0,2015-01-09 10:18:00+05:30,8280.70,8283.35,8279.40,8283.35
1,2015-01-09 10:19:00+05:30,8283.05,8283.70,8281.05,8283.35
2,2015-01-09 10:20:00+05:30,8283.65,8284.15,8283.10,8283.95
3,2015-01-09 10:21:00+05:30,8283.75,8285.75,8283.60,8285.75
4,2015-01-09 10:22:00+05:30,8286.20,8287.70,8285.75,8286.80


In [8]:
data["Date"] =  data["date"].str.split(" ").str[0]

In [9]:
data["Time"] =  data["date"].str.split(" ").str[1].str.split("+").str[0]

In [10]:
data["Date_Time"] = data['Date'] + " " + data["Time"]

In [11]:
data.drop("Date", axis = 1, inplace = True)

In [12]:
data

,date,open,high,low,close,Time,Date_Time
0,2015-01-09 10:18:00+05:30,8280.70,8283.35,8279.40,8283.35,10:18:00,2015-01-09 10:18:00
1,2015-01-09 10:19:00+05:30,8283.05,8283.70,8281.05,8283.35,10:19:00,2015-01-09 10:19:00
2,2015-01-09 10:20:00+05:30,8283.65,8284.15,8283.10,8283.95,10:20:00,2015-01-09 10:20:00
3,2015-01-09 10:21:00+05:30,8283.75,8285.75,8283.60,8285.75,10:21:00,2015-01-09 10:21:00
4,2015-01-09 10:22:00+05:30,8286.20,8287.70,8285.75,8286.80,10:22:00,2015-01-09 10:22:00
...,...,...,...,...,...,...,...
666330,2022-10-24 19:10:00+05:30,17732.10,17734.55,17729.75,17734.30,19:10:00,2022-10-24 19:10:00
666331,2022-10-24 19:11:00+05:30,17733.40,17736.10,17732.40,17735.15,19:11:00,2022-10-24 19:11:00
666332,2022-10-24 19:12:00+05:30,17734.55,17736.50,17732.20,17733.80,19:12:00,2022-10-24 19:12:00
666333,2022-10-24 19:13:00+05:30,17733.75,17738.60,17733.75,17736.35,19:13:00,2022-10-24 19:13:00


In [13]:
data["Date_Time"] = pd.to_datetime(data["Date_Time"], format = "%Y-%m-%d %H:%M:%S")

In [14]:
data.index = data['Date_Time']

In [15]:
data.drop(["date"], axis = 1, inplace = True)

In [16]:
data["Next Date_Time"] = data["Date_Time"].shift(-1)

In [17]:
data

,open,high,low,close,Time,Date_Time,Next Date_Time
Date_Time,,,,,,,
2015-01-09 10:18:00,8280.70,8283.35,8279.40,8283.35,10:18:00,2015-01-09 10:18:00,2015-01-09 10:19:00
2015-01-09 10:19:00,8283.05,8283.70,8281.05,8283.35,10:19:00,2015-01-09 10:19:00,2015-01-09 10:20:00
2015-01-09 10:20:00,8283.65,8284.15,8283.10,8283.95,10:20:00,2015-01-09 10:20:00,2015-01-09 10:21:00
2015-01-09 10:21:00,8283.75,8285.75,8283.60,8285.75,10:21:00,2015-01-09 10:21:00,2015-01-09 10:22:00
2015-01-09 10:22:00,8286.20,8287.70,8285.75,8286.80,10:22:00,2015-01-09 10:22:00,2015-01-09 10:23:00
...,...,...,...,...,...,...,...
2022-10-24 19:10:00,17732.10,17734.55,17729.75,17734.30,19:10:00,2022-10-24 19:10:00,2022-10-24 19:11:00
2022-10-24 19:11:00,17733.40,17736.10,17732.40,17735.15,19:11:00,2022-10-24 19:11:00,2022-10-24 19:12:00
2022-10-24 19:12:00,17734.55,17736.50,17732.20,17733.80,19:12:00,2022-10-24 19:12:00,2022-10-24 19:13:00


# Converting Data from 1 min Time Frame to 5 min and 15 min

In [18]:
def convert_timeframe(data, tf):
    data = data.copy() #  To avoid modifying the original DataFrame
    
    new_data = {
        "Date_Time" : [],
        "Open" : [],
        "High" : [],
        "Low" : [],
        "Close" : []
    }

    step = 0

    for i in data.index:


        step += 1
        if step == 1:
            o = data.loc[i, "open"]
            t = i
            h = data.loc[i, "high"]
            l = data.loc[i, "low"]

        else:
            if data.loc[i, "high"]>h:
                h = data.loc[i, "high"]

            if data.loc[i, "low"] < l:
                l = data.loc[i, "low"]

        if step == tf or  i.day!= data.loc[i, "Next Date_Time"].day:
            step = 0
            c = data.loc[i, "close"]

            new_data["Date_Time"].append(t)
            new_data["Open"].append(o)
            new_data["High"].append(h)
            new_data["Low"].append(l)
            new_data["Close"].append(c)
    return pd.DataFrame(new_data)

In [19]:
data15 = convert_timeframe(data, 15)

In [20]:
data5 = convert_timeframe(data, 5)

In [21]:
data5_original = data5.copy()

In [22]:
data15_original = data15.copy()

In [23]:
data15

,Date_Time,Open,High,Low,Close
0,2015-01-09 10:18:00,8280.70,8291.45,8279.40,8288.55
1,2015-01-09 10:33:00,8288.45,8290.00,8279.50,8281.05
2,2015-01-09 10:48:00,8281.15,8288.85,8279.10,8284.95
3,2015-01-09 11:03:00,8283.30,8288.30,8278.15,8278.50
4,2015-01-09 11:18:00,8278.30,8280.70,8250.20,8252.30
...,...,...,...,...,...
44424,2022-10-21 15:15:00,17571.35,17596.75,17570.75,17583.75
44425,2022-10-24 18:15:00,17736.35,17776.50,17733.45,17740.70
44426,2022-10-24 18:30:00,17739.55,17744.35,17725.40,17734.90
44427,2022-10-24 18:45:00,17735.35,17738.20,17728.90,17733.65


In [24]:
data5

,Date_Time,Open,High,Low,Close
0,2015-01-09 10:18:00,8280.70,8287.70,8279.40,8286.80
1,2015-01-09 10:23:00,8286.60,8288.65,8285.25,8288.40
2,2015-01-09 10:28:00,8288.25,8291.45,8286.30,8288.55
3,2015-01-09 10:33:00,8288.45,8290.00,8283.85,8286.15
4,2015-01-09 10:38:00,8286.20,8287.45,8282.55,8283.45
...,...,...,...,...,...
133271,2022-10-24 18:50:00,17733.80,17736.40,17731.35,17732.65
133272,2022-10-24 18:55:00,17733.55,17734.20,17728.90,17733.65
133273,2022-10-24 19:00:00,17733.10,17733.85,17716.35,17718.35
133274,2022-10-24 19:05:00,17719.90,17734.10,17708.40,17730.75


# Testing 5 EMA Strategy on 5 min interval (Short Strategy) 

In [25]:
status = None
alert_candle = None
position = None
short_at = None
stop_loss = None

data5 = data5_original.copy()
data5["5EMA"] = ta.trend.EMAIndicator(data5["Close"], 5).ema_indicator()
day = None

for i in data5.index:

    if position == None:
    
        if alert_candle == None: # Previous Candle is not an alert candle
                
            if data5.loc[i, "5EMA"] < data5.loc[i, "Low"]: # Alert Candle
                alert_candle = i
                data5.loc[i, "ALERT"] = "ALERT"
        

        else: # Previous Candle is an alert candle -> Try to take postion 
            
            
            if data5.loc[i, "Low"] < data5.loc[alert_candle, "Low"]:
                position = "SHORT"
                data5.loc[i, "POSITION"] = "ENTRY"

                if data5.loc[i, "Date_Time"].day != day:
                    data5.loc[i, "Days fund required"] = 1
                    day = data5.loc[i, "Date_Time"].day

                short_at = data5.loc[alert_candle, "Low"]

                stop_loss = min(data5.loc[alert_candle, "High"], short_at + MAX_SHORT_SL_POINTS)
                original_sl = stop_loss
                target = short_at - 3*(stop_loss - short_at)

                data5.loc[i, "TARGET"] = target
                data5.loc[i, "SL"] = stop_loss
                data5.loc[i, "ENTRY_AT"] = short_at
                alert_candle = None

            elif data5.loc[i, "5EMA"] < data5.loc[i, "Low"]: # Check for once again alert candle
                alert_candle = i
                data5.loc[i, "ALERT"] = "ALERT"
            else:
                alert_candle = None

            
                
    
    else: # I already have a short position

        
        data5.loc[i, "POSITION"] = "HOLD"
        data5.loc[i, "SL"] = stop_loss
        data5.loc[i, "TARGET"] = target
        data5.loc[i, "ENTRY_AT"] = short_at
        
        if data5.loc[i, "Low"] <= target:
            data5.loc[i, "POSITION"] = "TRAIL"
            
            temp = target - (original_sl - short_at)
            stop_loss = target
            target = temp
            data5.loc[i, "SL"] = stop_loss
            data5.loc[i, "TARGET"] = target
        
        elif data5.loc[i, "High"] >= stop_loss:
            data5.loc[i, "POSITION"] = "EXIT"
            position = None
            profit = short_at - stop_loss
            data5.loc[i, "PROFIT"] = profit
            data5.loc[i, "EXIT_AT"] = stop_loss

In [26]:
data5.iloc[4222:4267] # Lets check any two random interval

,Date_Time,Open,High,Low,Close,5EMA,ALERT,POSITION,Days fund required,TARGET,SL,ENTRY_AT,PROFIT,EXIT_AT
4222,2015-04-09 12:55:00,8742.45,8745.50,8740.05,8743.95,8741.811340,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4223,2015-04-09 13:00:00,8744.15,8750.30,8742.35,8747.20,8743.607560,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4224,2015-04-09 13:05:00,8746.70,8748.20,8742.65,8743.35,8743.521707,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4225,2015-04-09 13:10:00,8743.30,8746.75,8741.90,8744.75,8743.931138,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4226,2015-04-09 13:15:00,8744.85,8747.75,8743.85,8745.40,8744.420759,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4227,2015-04-09 13:20:00,8745.85,8753.05,8745.20,8751.90,8746.913839,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4228,2015-04-09 13:25:00,8751.90,8757.00,8750.40,8756.20,8750.009226,ALERT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4229,2015-04-09 13:30:00,8755.45,8764.50,8755.30,8762.35,8754.122817,ALERT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4230,2015-04-09 13:35:00,8761.20,8762.10,8752.45,8752.65,8753.631878,NaN,ENTRY,NaN,8737.30,8761.30,8755.30,NaN,NaN
4231,2015-04-09 13:40:00,8752.20,8753.75,8743.85,8750.70,8752.654585,NaN,HOLD,NaN,8737.30,8761.30,8755.30,NaN,NaN


In [27]:
data5.index = data5["Date_Time"]

In [28]:
data5["PROFIT"].resample("A").sum()

Date_Time
2015-12-31     4669.25
2016-12-31     4669.90
2017-12-31     2573.80
2018-12-31     4427.30
2019-12-31     6021.50
2020-12-31     7884.55
2021-12-31    10409.85
2022-12-31    11907.85
Freq: A-DEC, Name: PROFIT, dtype: float64

# Testing 5 EMA Strategy on 15 min interval (Long Strategy) 

In [29]:
status = None
alert_candle = None
position = None
long_at = None
stop_loss = None

data15 = data15_original.copy()
data15["5EMA"] = ta.trend.EMAIndicator(data15["Close"], 5).ema_indicator()
day = None

for i in data15.index:

        
    if position == None:
    
        if alert_candle == None: # Previous Candle is not an alert candle
                
            if data15.loc[i, "5EMA"] > data15.loc[i, "High"]: # Alert Candle
                alert_candle = i
                data15.loc[i, "ALERT"] = "ALERT"
            

        else: # Previous Candle is an alert candle -> Try to take postion 
            
            
            if data15.loc[i, "High"] > data15.loc[alert_candle, "High"]:
                position = "LONG"
                data15.loc[i, "POSITION"] = "ENTRY"

                if data15.loc[i, "Date_Time"].day != day:
                    data15.loc[i, "Days fund required"] = 1
                    day = data15.loc[i, "Date_Time"].day

                long_at = data15.loc[alert_candle, "High"]

                stop_loss = max(data15.loc[alert_candle, "Low"], long_at - MAX_LONG_SL_POINTS)
                original_sl = stop_loss
                target = long_at + 3*(long_at - stop_loss)

                data15.loc[i, "TARGET"] = target
                data15.loc[i, "SL"] = stop_loss
                data15.loc[i, "ENTRY_AT"] = long_at
                alert_candle = None

            elif data15.loc[i, "5EMA"] > data15.loc[i, "High"]: # Check for once again alert candle
                alert_candle = i
                data15.loc[i, "ALERT"] = "ALERT"
            else:
                alert_candle = None

            
                
    
    else: # I already have a position

        
        data15.loc[i, "POSITION"] = "HOLD"
        data15.loc[i, "SL"] = stop_loss
        data15.loc[i, "TARGET"] = target
        data15.loc[i, "ENTRY_AT"] = long_at
        
        if data15.loc[i, "High"] >= target:
            data15.loc[i, "POSITION"] = "TRAIL"
            
            temp = target + (long_at - original_sl)
            stop_loss = target
            target = temp
            data15.loc[i, "SL"] = stop_loss
            data15.loc[i, "TARGET"] = target
        
        elif data15.loc[i, "Low"] <= stop_loss:
            data15.loc[i, "POSITION"] = "EXIT"
            position = None
            profit = stop_loss - long_at
            data15.loc[i, "PROFIT"] = profit
            data15.loc[i, "EXIT_AT"] = stop_loss

In [30]:
data15.loc[4353:4373] # Lets check any two random interval

,Date_Time,Open,High,Low,Close,5EMA,ALERT,POSITION,Days fund required,TARGET,SL,ENTRY_AT,PROFIT,EXIT_AT
4353,2015-10-13 11:30:00,8123.25,8123.25,8113.70,8117.85,8119.621122,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4354,2015-10-13 11:45:00,8117.80,8123.45,8112.00,8123.45,8120.897415,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4355,2015-10-13 12:00:00,8123.50,8130.70,8121.25,8123.45,8121.748276,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4356,2015-10-13 12:15:00,8123.50,8140.65,8123.50,8140.60,8128.032184,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4357,2015-10-13 12:30:00,8140.85,8141.45,8121.55,8122.70,8126.254790,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4358,2015-10-13 12:45:00,8122.70,8122.70,8111.05,8117.40,8123.303193,ALERT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4359,2015-10-13 13:00:00,8117.55,8118.45,8108.35,8117.15,8121.252129,ALERT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4360,2015-10-13 13:15:00,8117.20,8117.35,8107.25,8107.25,8116.584752,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4361,2015-10-13 13:30:00,8107.00,8107.40,8095.85,8101.55,8111.573168,ALERT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4362,2015-10-13 13:45:00,8101.80,8102.65,8088.95,8098.10,8107.082112,ALERT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
data15.index = data15["Date_Time"]

In [32]:
data15["PROFIT"].resample("A").sum()

Date_Time
2015-12-31    2177.45
2016-12-31    2819.75
2017-12-31    2052.40
2018-12-31    2270.00
2019-12-31    3007.10
2020-12-31    4535.60
2021-12-31    6238.15
2022-12-31    7320.00
Freq: A-DEC, Name: PROFIT, dtype: float64

# Combining the Long and Short Strategy and applying them together.

In [33]:
test_data5 = data5[["Date_Time", "POSITION", "PROFIT"]]

In [34]:
test_data15 = data15[["Date_Time", "POSITION", "PROFIT"]]

In [35]:
data15i = 0
data5i = 0
position = None
final_data = {
    "Entry_Date_Time" : [],
    "Position" : [],
    "Exit_Date_Time": [],
    "Profit" : []
}

def calculate_profit(i5, i15):
    global position, final_data, test_data15, test_data5
    
    if position == None:
        
        if test_data15.iloc[i15, 1] == "ENTRY":
            position = "LONG"
            final_data["Entry_Date_Time"].append(test_data15.iloc[i15, 0])
            final_data["Position"].append("Long")
        
        elif test_data5.iloc[i5, 1] == "ENTRY":
            position = "SHORT"
            final_data["Entry_Date_Time"].append(test_data5.iloc[i5, 0])
            final_data["Position"].append("Short")
    
    elif position == "LONG":
         if test_data15.iloc[i15, 1] == "EXIT":
                final_data["Exit_Date_Time"].append(test_data15.iloc[i15, 0])
                final_data["Profit"].append(test_data15.iloc[i15, 2])
                position = None

    elif position == "SHORT":
         if test_data5.iloc[i5, 1] == "EXIT":
                final_data["Exit_Date_Time"].append(test_data5.iloc[i5, 0])
                final_data["Profit"].append(test_data5.iloc[i5, 2])
                position = None

while True:
    
    if data15i == len(test_data15):
        break
    if data15.iloc[data15i, 0] > data5.iloc[data5i, 0]:
        calculate_profit(data5i, data15i)
        data5i += 1
    else:
        calculate_profit(data5i, data15i)
        data15i += 1

In [36]:
final_data["Exit_Date_Time"].append(None)
final_data["Profit"].append(None)

In [37]:
final_data = pd.DataFrame(final_data)

In [38]:
final_data

,Entry_Date_Time,Position,Exit_Date_Time,Profit
0,2015-01-09 12:48:00,Long,2015-01-09 13:18:00,36.0
1,2015-01-09 13:13:00,Short,2015-01-09 13:28:00,24.0
2,2015-01-09 14:58:00,Short,2015-01-09 15:03:00,-6.0
3,2015-01-09 15:13:00,Short,2015-01-12 09:25:00,24.0
4,2015-01-12 10:05:00,Short,2015-01-12 10:15:00,-6.0
...,...,...,...,...
9500,2022-10-20 15:25:00,Short,2022-10-21 09:15:00,-6.0
9501,2022-10-21 09:35:00,Short,2022-10-21 09:40:00,-6.0
9502,2022-10-21 10:50:00,Short,2022-10-21 11:25:00,-6.0
9503,2022-10-21 11:45:00,Short,2022-10-21 12:00:00,24.0


In [39]:
final_data.index = final_data["Entry_Date_Time"]

In [40]:
final_data.resample("A").sum()

,Profit
Entry_Date_Time,
2015-12-31,5646.80
2016-12-31,6073.90
2017-12-31,3675.45
2018-12-31,6523.20
2019-12-31,8287.00
2020-12-31,11211.40
2021-12-31,14554.15
2022-12-31,15652.25


# Conclusion: It is evident that the profits from the aforementioned strategy have shown consistent improvement year after year. However, it is important to note that the data for the year 2022 was only available until October, thereby not providing a comprehensive picture of the annual profits of 2022

In [41]:
final_data[final_data["Profit"] < 0].resample("A").count()["Profit"] # Stop Loss

Entry_Date_Time
2015-12-31    764
2016-12-31    733
2017-12-31    749
2018-12-31    812
2019-12-31    819
2020-12-31    932
2021-12-31    904
2022-12-31    703
Freq: A-DEC, Name: Profit, dtype: int64

In [42]:
final_data[final_data["Profit"] > 0].resample("A").count()["Profit"] # Target

Entry_Date_Time
2015-12-31    343
2016-12-31    372
2017-12-31    326
2018-12-31    369
2019-12-31    402
2020-12-31    414
2021-12-31    460
2022-12-31    400
Freq: A-DEC, Name: Profit, dtype: int64

# Conclusion: We can see that there are twice the number of Stop Loss hits than the number of taregets achived. 

In [43]:
data15["Range"] = np.where(data15["High"] > data15["Low"], data15["High"] - data15["Low"],  data15["Low"] -  data15["High"])

In [44]:
np.mean(data15["Range"])

23.241880303405456

In [45]:
data5["Range"] = np.where(data5["High"] > data5["Low"], data5["High"] - data5["Low"],  data5["Low"] -  data5["High"])

In [46]:
np.mean(data5["Range"])

12.740139635042903

# On average, Nifty moves by 12 points within a span of 5 minutes and by 23 points within 15 minutes. Consequently, I have chosen 6 (half of 12) as the maximum Stop Loss that I am willing to tolerate on the short side, and approximately 12 (approx. half of 23) as the maximum Stop Loss I can bear on the long side.

In [51]:
data15[data15["Date_Time"].dt.year == 2022]["Range"].describe()

count    4679.000000
mean       38.229697
std        22.124240
min         8.900000
25%        23.600000
50%        32.800000
75%        46.300000
max       389.050000
Name: Range, dtype: float64

In [52]:
data5[data5["Date_Time"].dt.year == 2022]["Range"].mean()

20.914226686613812

In [53]:
data5[data5["Date_Time"].dt.year == 2022]["Range"].describe()

count    14037.000000
mean        20.914227
std         13.120197
min          0.000000
25%         12.700000
50%         17.700000
75%         25.450000
max        389.050000
Name: Range, dtype: float64

In [54]:
data5[data5["Date_Time"].dt.year == 2022]["Range"].mean()

20.914226686613812